In [14]:
from ultralytics import YOLO
import torch.nn as nn

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

# YOLO의 Detect 모듈 수정: 마지막 레이어를 5개의 파라미터 (cx, cy, w, h, angle)로 확장
class RotatedYOLO(nn.Module):
    def __init__(self, base_model):
        super(RotatedYOLO, self).__init__()
        self.base_model = base_model

        # YOLO의 Detect 레이어 가져오기
        self.detect_layer = self.base_model.model.model[-1]

        # YOLO의 기존 클래스 개수
        num_classes = self.detect_layer.nc

        # 새로운 Conv 레이어 추가: (cx, cy, w, h, angle) + obj_conf + class_probs
        in_channels = self.detect_layer.cv3[-1][2].out_channels  # 마지막 Conv 레이어의 출력 채널
        out_channels = 5 + 1 + num_classes  # (cx, cy, w, h, angle, obj_conf + class_probs)
        
        # 새로운 Conv2d 레이어 생성
        self.new_conv = nn.Conv2d(in_channels, out_channels, kernel_size=(1, 1))

        # Detect 레이어의 cv3 수정
        self.detect_layer.cv3[-1].add_module('new_conv', self.new_conv)

    def forward(self, x):
        # YOLO의 forward 과정을 수행
        return self.base_model(x)

    def train(self, *args, **kwargs):
        # YOLO의 train 메소드 호출
        return self.base_model.train(*args, **kwargs)

# 기존 모델에 수정된 구조 적용
rotated_model = RotatedYOLO(model)

# 커스텀 손실 함수 정의
def custom_loss(preds, targets):
    # 바운딩 박스 손실 계산 (cx, cy, w, h)
    bbox_loss = nn.MSELoss()(preds[:, :4], targets[:, :4])  # (cx, cy, w, h)

    # 각도 손실 계산 (angle)
    angle_loss = nn.MSELoss()(preds[:, 4], targets[:, 4])  # angle

    # 총 손실 계산 (기존 바운딩 박스 손실 + 각도 손실)
    total_loss = bbox_loss + 0.1 * angle_loss  # 0.1은 각도 손실 가중치
    return total_loss

# 학습 시작 (rotated_model을 사용하여 훈련)
results = rotated_model.train(
    data='data.yaml',  # 데이터 설정 파일
    epochs=50,         # 학습 epoch 수
    imgsz=640,         # 입력 이미지 크기
    batch=16,          # 배치 크기
    name='rotated_ship_detection',  # 실험 이름
    device=0           # GPU 사용 설정
)


Ultralytics 8.3.4  Python-3.11.4 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11m.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=rotated_ship_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pook0612 (limbw). Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed 


train: Scanning D:\Workspace\SatellieteIMGDetection\train\img.cache... 0 images, 3851 backgrounds, 0 corrupt: 100%|██████████| 3851/3851 [00:00<?, ?it/s]

WARNING  No labels found in D:\Workspace\SatellieteIMGDetection\train\img.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\Workspace\SatellieteIMGDetection\valid\img.cache... 0 images, 475 backgrounds, 0 corrupt: 100%|██████████| 475/475 [00:00<?, ?it/s]

WARNING  No labels found in D:\Workspace\SatellieteIMGDetection\valid\img.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


In [7]:
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

# 모델의 전체 구조를 요약해서 출력
print(model.model)


DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   

In [8]:

# 모델의 레이어 이름과 모듈을 모두 출력
for name, module in model.model.named_modules():
    print(f"Layer Name: {name}, Module: {module}")


Layer Name: , Module: DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): 

In [10]:
# YOLO의 Detect 모듈 출력
detect_layer = model.model.model[-1]  # Detect 레이어에 접근
print(detect_layer)


Detect(
  (cv2): ModuleList(
    (0): Sequential(
      (0): Conv(
        (conv): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1-2): 2 x Sequential(
      (0): Conv(
        (conv): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)